# Data Collection Notebook for Texas ... 

## Import Libraries

In [1]:
#Import requisite modules
import sys
import pandas as pd
import numpy as np
import datetime
import os #Library module for .csv file check
from twitterscraper import query_tweets #if you haven't installed this module, run 'pip install twitterscraper' in your notebook or terminal


INFO: {'User-Agent': 'Mozilla/5.0 (Windows NT 5.2; RW; rv:7.0a1) Gecko/20091211 SeaMonkey/9.23a1pre'}


## Data Collection 

In [2]:
# credit for the following code -- Adam Cohen, Najiha Boosra, Max Mazel of  DSI-CC11-NYC 
# https://github.com/Adam395/Twitter-Scraper

#Get tweets without geolocation
def get_tweets(query, year, month, day): 
    tweets = {} #Instantiates a new empty dictionary
    count = 0 #Instantiates an index generator
    for tweet in query_tweets(query,begindate=datetime.date(year,month,day)):
        chirp = {} #Instantiates a new empty dictionary for each pulled tweet
        chirp['tweet_id'] = tweet.tweet_id
        chirp['username'] = tweet.username
        chirp['text'] = tweet.text
        chirp['tweet_date'] = tweet.timestamp
        chirp['search_term'] = query
        chirp['city'] = np.NaN #Fills columns with NaNs for data cleaning at a later point. These items are NaNs so that
        chirp['lat'] = np.NaN #They will be flagged in an EDA search for missing values, instead of being strings with no information
        chirp['long'] = np.NaN
        chirp['radius'] = np.NaN
        chirp['query_start'] = pd.to_datetime(f"{year}/{month}/{day}") #Generates date as datetime object
        tweets.update({count : chirp}) #Sets count value to tweets keys, so that index is automatically generated 
        count += 1 #increments index up by 1
    return tweets
#--------------------------------------------------------------------

#Get tweets with geolocation
def get_tweets_geoloc(query, city, lat, long, radius, year, month, day): #Geolocation parameters defined by user in master function or dictionary
    tweets = {}
    count = 0
    for tweet in query_tweets(f"{query}, geocode:{lat},{long},{radius}",begindate=datetime.date(year,month,day)):
        chirp = {} #Generates tweet dictionary by calling on generated 'tweet' object attributes
        chirp['tweet_id'] = tweet.tweet_id
        chirp['username'] = tweet.username
        chirp['text'] = tweet.text
        chirp['tweet_date'] = tweet.timestamp
        chirp['search_term'] = query
        chirp['city'] = city
        chirp['lat'] = lat
        chirp['long'] = long
        chirp['radius'] = radius
        chirp['query_start'] = pd.to_datetime(f"{year}/{month}/{day}") #Generates date as datetime object
        tweets.update({count : chirp}) #Sets count value to tweets keys, so that index is automatically generated 
        count += 1 #increments index up by 1
    return tweets
#--------------------------------------------------------------------

#Generate dataframe from "tweets" dictionary generated after each query
def make_dataframe(dictionary):
    df = pd.DataFrame.from_dict(dictionary, orient='index') #Creates a dataframe from the input dictionary 'tweets' later in function
    return df #creates a temporary dataframe for concatenation later
#--------------------------------------------------------------------

#Query function using custom parameters
def get_query_dataframe_cp(custom_params):
    query_df = pd.DataFrame() #instantiate an empty dataframe
    for key in custom_params.keys(): #Generates a new query dataframe for each city used in the parameter dictionary
        for query in custom_params[key]['queries']: #Runs a unique query for each unique term in the query key
            tweets = get_tweets_geoloc(query,custom_params[key],custom_params[key]['lat'],custom_params[key]['long'],
                                       custom_params[key]['radius'],custom_params[key]['start_year'],custom_params[key]['start_month'],
                                       custom_params[key]['start_day'])
            df = make_dataframe(tweets) #creates temporary dataframe from independent query
            query_df = pd.concat([query_df,df],ignore_index = True) #concatenates temporary dataframe 'df' to master query dataframe 
    return query_df
#Credit Danielle Medellin for the majority of the code block section
#---------------------------------------------------------------

#Query function with geolocation but no custom parameters
def get_query_dataframe_geo(list_of_queries,city,lat,long,radius,year,month,day):
    query_df = pd.DataFrame()
    for query in list_of_queries: #Runs the same functions as custom parameters code
            tweets = get_tweets_geoloc(query,city,lat,long,radius,year,month,day)
            df = make_dataframe(tweets)
            query_df = pd.concat([query_df,df],ignore_index = True)
    return query_df
#-------------------------------------------------------------------

#Query function with no custom anything
def get_query_dataframe(list_of_queries,year,month,day):
    query_df = pd.DataFrame()
    for query in list_of_queries:
            tweets = get_tweets(query,year,month,day)
            df = make_dataframe(tweets)
            query_df = pd.concat([query_df,df],ignore_index = True)
    return query_df
#------------------------------------------------------------------

#Master function
def get_dataset():   
    #Main function switches
    custom_params_switch = input("Are you using a custom parameter dictionary? (y/n)") #Enables or disables custom dictionary use in query parameter switch block
    if str.lower(custom_params_switch) != 'y': #If the user is not using parameters
        
        #Geolocation parameter configuration
        geo_switch = input("Are you using geolocation? (y/n)") #If no custom dictionary, enables or disables geolocation in query parameter switch block
        if str.lower(geo_switch) == 'y':
            lat = float(input("Input Latitude:")) #Converts string input latitude to float value
            long = float(input("Input Longitude:"))
            city = input("Input city or neighborhood corresponding to coordinates:") #Allows filling of city values
            radius = input("Input radius and unit:")
            
        #Date parameter configuration
        use_custom_start_date = input("Using custom query start? (y/n)") #Enables or disables custom date parameter
        if str.lower(use_custom_start_date) == 'y':
            year = int(input("Input start year YYYY:"))
            month = int(input("Input start month MM:"))
            day = int(input("Input start day DD:"))
        else: #Sets default date
            
            #Default date parameters. Users can change defaults here
            year = 2019
            month = 12
            day = 1
 
    #CSV export parameter configuration
    export_csv_switch = input("Do you want to export the final dataframe to csv? (y/n)")
    if str.lower(export_csv_switch) == 'y': #Enables CSV switch block later if yes as well
        custom_csv_name = input("Input CSV export file name:") #Prompts user for file name
        if os.path.exists(f'datasets/{custom_csv_name}.csv') == True: #Check if file has already been created. If yes, prompt user to overwrite or make new file.
            overwrite_check = input ("File already exists--do you want to overwrite? (y/n)")
            if str.lower(overwrite_check) != 'y': #If user uses any value except y, the function will not proceed
                new_csv_name = custom_csv_name #creates new_csv_name variable = to old name
                while new_csv_name == custom_csv_name: #continues to reject file name until a unique name is created
                    new_csv_name = input("Input new output file name:")
                custom_csv_name = new_csv_name #Sets the file name to the new user input
    
    #Query parameter switch block
    if str.lower(custom_params_switch) == 'y':
        dataset = get_query_dataframe_cp(custom_params)
    else:
        if str.lower(geo_switch) == 'y':
            dataset = get_query_dataframe_geo(query_list,city,lat,long,radius,year,month,day)
        else:
            dataset = get_query_dataframe(query_list,year,month,day)
    
    #CSV export switch block
    if str.lower(export_csv_switch) == 'y':
        if os.path.exists('datasets') != True:
            os.mkdir('datasets')
        dataset.to_csv(f"./datasets/{custom_csv_name}.csv", index = False) #write csv to datasets folder
        print(f"Export complete, scraped {len(dataset.index)} tweets") #Prints completion statement including total tweets scraped       
    else:
        return dataset

### Texas - Custom Parameters

In [4]:
texas_query_list = [
    'power outage',
    'power out',
    'transformer AND power',
    'power cut',
    'power failure',
    'power interruption',
    'service interruption',
    'lights out',
    'electric out',
    'power off',
    'electric off',
    'lost power',
    '#poweroutage',
    'atmos energy AND outage',
    'atmosenergy AND outage',
    'ambit energy AND outage',
    'ambit energy AND power outage',
    'CNPalerts AND power outage',
    'centerpoint energy AND outage',
    'centerpoint energy AND power',
    'entergy texas AND outage',
    'entergy texas AND power'
]
custom_params = {'Houston':{'city' : 'Houston',
                         'lat'  : 29.760427,
                         'long' : -95.369804,
                         'radius': '15mi',
                         'queries' : texas_query_list,
                           'start_year': 2016,
                         'start_month': 1,
                         'start_day': 1},
              'San Antonio': {'city' : 'San Antonio',
                          'lat'  : 29.424122,
                          'long' : -98.493629,
                          'radius' : '15mi',
                         'queries': texas_query_list,
                             'start_year': 2016,
                         'start_month': 1,
                         'start_day': 1},
                 'Dallas':{'city' : 'Dallas',
                          'lat'  : 32.776665,
                          'long' : -96.796989,
                          'radius' : '15mi',
                         'queries': texas_query_list,
                          'start_year': 2016,
                         'start_month': 1,
                         'start_day': 1}
             }

In [ ]:
get_dataset()

Are you using a custom parameter dictionary? (y/n) y
Do you want to export the final dataframe to csv? (y/n) y
Input CSV export file name: texas_data


INFO: queries: ['power outage, geocode:29.760427,-95.369804,15mi since:2016-01-01 until:2016-03-20', 'power outage, geocode:29.760427,-95.369804,15mi since:2016-03-20 until:2016-06-08', 'power outage, geocode:29.760427,-95.369804,15mi since:2016-06-08 until:2016-08-26', 'power outage, geocode:29.760427,-95.369804,15mi since:2016-08-26 until:2016-11-14', 'power outage, geocode:29.760427,-95.369804,15mi since:2016-11-14 until:2017-02-02', 'power outage, geocode:29.760427,-95.369804,15mi since:2017-02-02 until:2017-04-22', 'power outage, geocode:29.760427,-95.369804,15mi since:2017-04-22 until:2017-07-11', 'power outage, geocode:29.760427,-95.369804,15mi since:2017-07-11 until:2017-09-28', 'power outage, geocode:29.760427,-95.369804,15mi since:2017-09-28 until:2017-12-17', 'power outage, geocode:29.760427,-95.369804,15mi since:2017-12-17 until:2018-03-07', 'power outage, geocode:29.760427,-95.369804,15mi since:2018-03-07 until:2018-05-25', 'power outage, geocode:29.760427,-95.369804,15mi 